In [13]:
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
import pandas as pd
from datetime import datetime
from google.oauth2.service_account import Credentials
import gspread
from google.cloud import storage
from datetime import datetime
from pathlib import Path
import asyncio

In [14]:
data = {
    "userid": [
    "10002536", "10008847", "10004679", "10023836",
    "10033752", "10038584", "10044023", "10192623",
    "10165926", "10093976", "10132431"
    ],
    "名前": [
        "キンシュウサイ", "大岡正樹", "池海龍（イケカイリュウ）", "飯島美桜",
        "江原ケイト", "樋口大輔", "加藤順平", "加藤奈津子",
        "松本明子", "石崎卓", "玉野利家"
    ],
    "PW": [
        "Kin20240301", "Hasegawa110", "Chihailong0803", "Cherry2525",
        "Tennis2784!", "46495963¥@Desu", "WCbifd@3NANwXGE", "aDA8a6MWs3XEz_Y",
        "Saikou1234", "Zaitaku2023", "Packen0731t"
    ]
}

user_df = pd.DataFrame(data)

COLUMNS = ["userid","予約番号", "利用日", "時間", "公園・施設", "設備予約", "支払状況"]

In [17]:
URL = "https://kouen.sports.metro.tokyo.lg.jp/web/index.jsp"
BLOCK_TEXT = "しばらく経ってから"
result_lst = []
async def scrape_cancel_list(user_df):
    p = await async_playwright().start()

    context = await p.chromium.launch_persistent_context(
        user_data_dir="./check_reservation",
        channel="chrome",
        headless=False,
    )
    page = await context.new_page()

    for _, row in user_df.iterrows():
        userid = str(row["userid"])
        pw = str(row["PW"])

        print(f"Processing user: {userid}")

        await page.goto(URL)

        # --- wait until not blocked ---
        while True:
            content = await page.content()
            if BLOCK_TEXT not in content:
                break
            print("Blocked. Waiting 5 seconds...")
            await asyncio.sleep(5)
            await page.reload()

        await page.click("text=ログイン")
        await page.fill("#userId", userid)
        await page.fill("#password", pw)

        # Login often triggers navigation
        async with page.expect_navigation(wait_until="networkidle"):
            await page.click("#btn-go")

        # 2) Open reservation modal (as you already did)
        await page.click('a[data-target="#modal-reservation-menus"]')
        await page.wait_for_selector("#modal-reservation-menus", state="visible")

        # 3) Trigger the cancel/reservation list action ONCE and wait for navigation (if it happens)
        try:
            async with page.expect_navigation(wait_until="networkidle", timeout=10000):
                await page.evaluate("doAction(document.form1, gRsvWGetCancelRsvDataAction)")
        except PlaywrightTimeoutError:
            # In some flows it may update without a full navigation
            await page.wait_for_load_state("networkidle")

        # 4) If no reservations, tbody won't exist -> skip
        tbody = await page.query_selector("#rsvacceptlist tbody")
        if not tbody:
            rows = [[userid,"", "", "", "", "", ""]]
            return pd.DataFrame(rows, columns=COLUMNS)

        # 5) Extract first 6 columns (0..5) from each row
        rows = await page.evaluate("""
        [...document.querySelectorAll('#rsvacceptlist tbody tr')]
            .map(tr => [...tr.querySelectorAll('td')]
            .slice(0, 6)
            .map(td => td.innerText.trim()))
            .filter(r => r.length === 6)
        """)
        rows = [[userid, *r] for r in rows]
        result_lst = result_lst.append(rows)
        await page.goto(URL)
    
    await context.close()
    await p.stop()
    return pd.concat(result_lst, ignore_index=True)
final_df = await scrape_cancel_list(user_df)


Processing user: 10002536


In [ ]:
managed_df = pd.read_excel("reservation_manage_sheet_python.xlsx", dtype=str)

df = pd.concat([managed_df, user_df], ignore_index=True)
df = df.apply(lambda col: col.str.strip())
df = df.apply(lambda c: c.astype(str).str.replace(r"[\r\n]", "", regex=True).str.strip())
df = df.astype(str)
df = df.drop_duplicates()

# Generate date string
today_str = datetime.today().strftime("%Y%m%d")

# Build path safely
base_path = Path.home() / "workspace" / "熊猫カンパニー" / "reservation"
base_path.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
file_path = base_path / f"reservations_{today_str}.xlsx"

# Export
df.to_excel(file_path, index=False)

import subprocess

subprocess.run(["paplay", "/usr/share/sounds/freedesktop/stereo/complete.oga"])

